# 2-clean&filter

In [62]:
import pandas as pd

df = pd.read_csv("../data/extract_16.csv", low_memory=False)
df.shape

(337030, 32)

In [63]:
df = df[df["Code_grammaire"] == "PAROLE_GENERIQUE"]

In [64]:
df.shape

(184905, 32)

In [65]:
df.head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction
3,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_1,NaN,Mme la présidente,NaN,719874.0,1007.69,La parole est à Mme la ministre de la transiti...,point
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre de la transition énergétique,759832.0,1041.01,"Après la promulgation, vendredi, de la loi rel...",point
6,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832.0,1162.28,Comment se dire écologiste lorsqu’on ne vote p...,point
8,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832.0,1173.04,…lorsqu’on vante le modèle allemand qui repose...,point
10,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832.0,1210.25,Si nous voulons atteindre la neutralité carbon...,point


In [66]:
# Exclure les prises de parole de "Mme la présidente" et "M. le président"
# col Role_debat : pas bien identifié en fait
# plutôt partir sur Nom_orateur

df = df[~df["Nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])]

In [67]:
df.shape

(129769, 32)

In [68]:
df["Texte"].str.len().describe()

count    129769.000000
mean        736.203816
std         930.668303
min           3.000000
25%         200.000000
50%         442.000000
75%         922.000000
max       21406.000000
Name: Texte, dtype: float64

In [69]:
df["len_dirtytext"] = df["Texte"].str.len()

In [70]:
df["Code_parole"].value_counts(dropna=False)

Code_parole
PAROLE_1_2       70643
NaN              38219
AVIS_COM_1_20    10951
AVIS_GVT_1_20     9954
AVIS_COM_1_10        1
PAROLE_1_1           1
Name: count, dtype: int64

In [71]:
df.groupby("Code_parole", dropna=False)["len_dirtytext"].describe()

,count,mean,std,min,25%,50%,75%,max
Code_parole,,,,,,,,
AVIS_COM_1_10,1.0,1278.000000,NaN,1278.0,1278.0,1278.0,1278.00,1278.0
AVIS_COM_1_20,10951.0,389.245457,416.340444,4.0,96.0,273.0,538.00,5395.0
AVIS_GVT_1_20,9954.0,418.768033,581.874887,4.0,20.0,212.0,581.75,6481.0
PAROLE_1_1,1.0,32.000000,NaN,32.0,32.0,32.0,32.00,32.0
PAROLE_1_2,70643.0,819.476325,1053.234730,3.0,221.0,464.0,1000.00,21406.0
NaN,38219.0,764.379314,823.602402,3.0,246.0,529.0,1008.00,16991.0


In [72]:
# voir ici pour aviser ce qu'il faut virer :
df[df["Code_parole"] == "AVIS_COM_1_20"]

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction,len_dirtytext
326,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990.0,8834.43,Avis défavorable. On ne va pas débattre pendan...,point,237
368,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990.0,10235.34,Je ne vous surprendrai pas : avis défavorable....,point,1048
433,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990.0,11406.98,Je ne sais pas si les énergies renouvelables f...,point,350
436,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990.0,11418.63,"Vous êtes aussi très nombreux, chers collègues...",point,361
454,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990.0,11788.01,Avis défavorable.,point,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336459,CRSANR5L16S2024O1N132,RUANR5L16S2024IDS28086,SCR5A2024O1,20240229150000000,jeudi 29 février 2024,2,132,AN,16,Session ordinaire 2023-2024,...,NORMAL,AVIS_COM_1_20,NaN,Mme Christine Pires Beaune,rapporteure,608172.0,16540.30,"J’émettrai un avis défavorable, mais je préfér...",point,520
336469,CRSANR5L16S2024O1N132,RUANR5L16S2024IDS28086,SCR5A2024O1,20240229150000000,jeudi 29 février 2024,2,132,AN,16,Session ordinaire 2023-2024,...,NORMAL,AVIS_COM_1_20,NaN,Mme Christine Pires Beaune,rapporteure,608172.0,16858.88,Avis défavorable que je vais tenter d’explique...,point,1277
336484,CRSANR5L16S2024O1N132,RUANR5L16S2024IDS28086,SCR5A2024O1,20240229150000000,jeudi 29 février 2024,2,132,AN,16,Session ordinaire 2023-2024,...,NORMAL,AVIS_COM_1_20,NaN,Mme Christine Pires Beaune,rapporteure,608172.0,17246.86,Je demande le retrait de l’amendement ; à défa...,point,630
336506,CRSANR5L16S2024O1N132,RUANR5L16S2024IDS28086,SCR5A2024O1,20240229150000000,jeudi 29 février 2024,2,132,AN,16,Session ordinaire 2023-2024,...,NORMAL,AVIS_COM_1_20,NaN,Mme Christine Pires Beaune,rapporteure,608172.0,17572.30,Je rappelle que le texte prévoit un plafonneme...,point,940


## Nettoyage des textes

Aller nettoyer les balises, etc.

In [73]:
df.to_csv("../data/data_cleaning.csv", index=False)